# Phase 2 Slack Experiment - GPU Accelerated

This notebook runs the Phase 2 Slack experiment on Kaggle's free GPU to analyze η/ε dynamics and Agent C behavior.

## Setup
1. Enable GPU: Settings → Accelerator → GPU T4 x2
2. Add dataset: Upload adjunction-model code as Kaggle dataset
3. Run all cells

In [ ]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

In [ ]:
# Install dependencies
!pip install torch-geometric torch-scatter torch-sparse -q
!pip install matplotlib seaborn pandas numpy -q

In [ ]:
# Clone or copy the adjunction-model repository
# Option 1: Clone from GitHub
!git clone https://github.com/type37c/adjunction-model.git
%cd adjunction-model

# Option 2: If uploaded as Kaggle dataset, copy from /kaggle/input/
# !cp -r /kaggle/input/adjunction-model/* .
# %cd /kaggle/working

In [ ]:
# Import necessary modules
import sys
sys.path.append('/kaggle/working/adjunction-model')

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path

from src.models.adjunction_model import AdjunctionModel
from src.training.train_phase2_slack import Phase2SlackTrainer
from src.data.shapenet_dataset import ShapeNetAffordanceDataset

print("✓ All imports successful")

In [ ]:
# Configuration
CONFIG = {
    'num_epochs': 100,
    'num_shapes': 100,  # Larger dataset on GPU
    'batch_size': 8,     # Larger batch size on GPU
    'num_points': 512,
    'num_affordances': 5,
    'learning_rate': 1e-4,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'output_dir': 'results/phase2_slack_gpu'
}

print(f"Device: {CONFIG['device']}")
print(f"Epochs: {CONFIG['num_epochs']}")
print(f"Shapes: {CONFIG['num_shapes']}")
print(f"Batch size: {CONFIG['batch_size']}")

In [ ]:
# Create output directory
output_dir = Path(CONFIG['output_dir'])
output_dir.mkdir(parents=True, exist_ok=True)
print(f"Output directory: {output_dir}")

In [ ]:
# Create dataset
print("Creating dataset...")
dataset = ShapeNetAffordanceDataset(
    num_shapes=CONFIG['num_shapes'],
    num_points=CONFIG['num_points'],
    num_affordances=CONFIG['num_affordances']
)

# Split into train/val
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

print(f"Train size: {len(train_dataset)}")
print(f"Val size: {len(val_dataset)}")

In [ ]:
# Create model
print("Creating model...")
model = AdjunctionModel(
    num_affordances=CONFIG['num_affordances'],
    num_points=CONFIG['num_points'],
    f_hidden_dim=64,
    g_hidden_dim=128,
    agent_hidden_dim=256,
    agent_latent_dim=64,
    context_dim=128
).to(CONFIG['device'])

total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")

In [ ]:
# Create trainer
print("Creating trainer...")
trainer = Phase2SlackTrainer(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    batch_size=CONFIG['batch_size'],
    learning_rate=CONFIG['learning_rate'],
    device=CONFIG['device'],
    output_dir=str(output_dir)
)

print("✓ Trainer ready")

In [ ]:
# Train
print("\n" + "="*50)
print("Starting training...")
print("="*50 + "\n")

trainer.train(num_epochs=CONFIG['num_epochs'])

print("\n" + "="*50)
print("Training completed!")
print("="*50)

In [ ]:
# Load and display metrics
with open(output_dir / 'metrics.json', 'r') as f:
    metrics = json.load(f)

print("\nFinal Metrics:")
print(f"  Unit η (final): {metrics['coherence_signal'][-1]:.4f}")
print(f"  Affordance Loss (final): {metrics['affordance_loss'][-1]:.4f}")
print(f"  KL Loss (final): {metrics['kl_loss'][-1]:.4f}")
print(f"  Coherence (final): {metrics['coherence'][-1]:.4f}")

In [ ]:
# Display visualizations
from IPython.display import Image, display

print("\nSlack Signals:")
display(Image(filename=str(output_dir / 'slack_signals.png')))

print("\nTraining Losses:")
display(Image(filename=str(output_dir / 'training_losses.png')))

In [ ]:
# Save results for download
print("\nResults saved to:", output_dir)
print("\nFiles:")
for f in sorted(output_dir.glob('*')):
    print(f"  - {f.name}")